In [2]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import torchaudio 
from torch.utils.data import Dataset, DataLoader
import os 
from pathlib import Path
from torchvision.utils import save_image
from PIL import Image
import torchvision.transforms as T


/home/dmbs/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 6050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:

classes = ['teens','twenties','thirties','fourties','fifties','sixties','seventies'] 

def load_audio_data (path, label):
    dataset=[]
    walker = sorted(str(p) for p in Path(path).glob(f'*.wav'))
    for i , file_path in enumerate (walker):
        data = dict()
        path, filename = os.path.split(file_path)
        speaker, _ = os.path.splitext (filename)
        _, speaker_id = speaker.split("_en_")
        waveform, sample_rate = torchaudio.load (file_path)
        data['waveform'] = waveform
        data['sample_rate'] = sample_rate
        data['label'] = label
        data['speaker_id'] = speaker_id


        dataset.append(data)
    return dataset
class MyDataset(Dataset):
    def __init__(self, label):
        super(MyDataset, self).__init__()
        self.data=load_audio_data(f'input/data/{label}', label)

    def __getitem__(self,item):
        return self.data[item]

    def __len__(self):
        return len(self.data)
# def dataloader (labels, batch_size):
#     dataloaders = dict()
#     for label in labels:
#          dataset = MyDataset(label)
#          dataloaders[label] = DataLoader(dataset = dataset, batch_size = batch_size, shuffle = True, collate_fn = lambda i: i, num_workers =1)
    #return dataloaders
if __name__ == "__main__":
    data = MyDataset('teens')
    waveform = data[2]['waveform']
    speaker_id = data[2]['speaker_id']
    print("shape of wave form , ", waveform.size)
    spectrogram_tensor = torchaudio.transforms.Spectrogram()(waveform)
    #save_image(spectrogram_tensor, 'input/data/Spectrograms/{}.png'.format(speaker_id))
    # print ("shape of the spectrogram: {}".format(spectrogram_tensor.size()))
    # fig= plt.figure()
    plt.figure()
    plt.imsave('input/data/{}.png',spectrogram_tensor.log2()[0,:,:].numpy(), vmin=-100, vmax= 0, cmap ='gray')
    # plt.imshow (spectrogram_tensor.log2()[0,:,:].numpy(), cmap='gray')
    # plt.show()
    # plt.savefig('input/data/{}.png'.format(speaker_id),bbox_inches= "tight", pad_inches =0.0)
    # plt.close(fig)
    #save_image(spectrogram_tensor,'input/data/{}.png'.format(speaker_id) )
   